In [41]:
from ipywidgets import interact, Dropdown, Output, GridspecLayout, VBox, BoundedIntText, HBox, Button, TwoByTwoLayout, Layout
import calculations

In [42]:
### Define the Devourer Level widgets

# Define the options for dropdown menus
options = {
    'Devourer Lvl': [x for x in range(35, 101, 5)]
}

# Define the output textbox
output_level = Output()

# Define a function to update the level textbox
def update_output_level(dev_level_dropdown_val):
    output_level.clear_output()
    with output_level:
        print(f"{calculations.get_level_shards(dev_level_dropdown_val):,}")    # :, automatically adds the thousands separator

# Create dropdown widgets
dev_level_dropdown = Dropdown(options=options['Devourer Lvl'], description='Devourer Lvl:')

# Create an interaction between the dropdowns and the output textbox
dev_level_dropdown.observe(update_output_level, names='value')

In [43]:
### Define the Feats widgets

# Define the options for dropdown menus
options = {
    'Feats': [x for x in range(1, 26)],
}

# Define the output textbox
output_feats = Output()

# Define a function to update the feats textbox
def update_output_feats(feats_dropdown_val):
    output_feats.clear_output()
    with output_feats:
        print(f"{calculations.get_feat_shards(feats_dropdown_val)}%")

# Create dropdown widgets
feats_dropdown = Dropdown(options=options['Feats'], description='Feats:')

# Create an interaction between the dropdowns and the output textbox
feats_dropdown.observe(update_output_feats, names='value')

In [44]:
### Define the Other widget

# Define the output textbox
output_other = Output()

# Define int input box for Other category
other_shards = BoundedIntText(value=100, min=100, max=150, description='Other:', disabled=False)

# Define a function to update the other textbox
def update_output_other(other_val):
    output_other.clear_output()
    with output_other:
        print(f"{other_val}%")

# Create an interaction between the dropdowns and the output textbox
other_shards.observe(update_output_other, names='value')

In [45]:
### Define the legendaries widget

# Define the output textbox
output_legend = Output()

# Define the options for dropdown menus
options = calculations.list_legendary_limits()

# Define the descriptions for dropdown menus
descriptions = calculations.list_legendaries()

# Create a 4x4 grid to hold the dropdowns
grid = GridspecLayout(4, 4)

# Create callback for grid changes
def cell_eventhandler(change):
    output_legend.clear_output()
    with output_legend:
        calculations.update_legendary_count(change['new'], change['owner'].description)
        print(calculations.get_legendary_shards())

# Create callback for clear button
def on_clear_click(click):
    calculations.init_legendary_table()
    reset_grid()
    

# Reset legendary counts to 0
def reset_grid():
    for i in range(4):
        for j in range(3):
            grid[i, j].value = 0

# Loop through each cell in the grid
for i in range(4):
    for j in range(3):
        idx = (3 * i) + j
        desc = descriptions[idx]
        
        if options[idx] == -1:
            textbox = BoundedIntText(value=0, min=0, max=100, description=f'{desc}', disabled=False)
            textbox.observe(cell_eventhandler, names='value')
            grid[i, j] = textbox
        else:
            dropdown = Dropdown(options=[x for x in range(0, options[idx] + 1)], description=f'{desc}')
            dropdown.observe(cell_eventhandler, names='value')
            grid[i, j] = dropdown

# Add output box and clear button in last column
clear_button = Button(description='Clear', disabled=False, layout=Layout(height='auto', width='auto'))
grid[:4, 3] = clear_button

# Add callback to button
clear_button.on_click(on_clear_click)

In [46]:
### Put it all together

level_and_feat = TwoByTwoLayout(
    top_left=dev_level_dropdown, 
    top_right=output_level,
    bottom_left=feats_dropdown,
    bottom_right=output_feats)
other = HBox([other_shards, output_other])

widget = VBox([dev_level_dropdown, feats_dropdown, grid, other_shards])
widget